<a href="https://colab.research.google.com/github/HedersonSantos/SmartHome/blob/main/seriesTemporais_sarimax_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 6.4 MB/s 
     |███████████████████████████     | 8.3 MB 29.3 MB/s eta 0:00:01

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

from sklearn.preprocessing import MinMaxScaler


from statsmodels.tsa.stattools import acovf, acf, pacf, pacf_yw, pacf_ols
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf # para determinar (p,q)
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.tools import diff
from statsmodels.tsa.stattools import adfuller

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.forecasting.theta import ThetaModel

from pmdarima.arima.utils import ndiffs
from pmdarima import auto_arima


from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler


In [ ]:
!wget https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/train.csv
!wget https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/test.csv
!wget https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/sample_submission.csv

In [ ]:
def plotaSeries(treino, teste, predito):
  if type(treino) == pd.core.series.Series:
    treino.plot(legend=True,label='Treino')
  if type(teste) == pd.core.series.Series:
    teste.plot(legend=True,label='Teste',figsize=(16,8))
  if  type(predito)==pd.core.series.Series:
    predito.plot(legend=True) 
  else:
    for p in predito.columns:
      predito[p].plot(legend=True)

In [ ]:
df_treino = pd.read_csv('train.csv')
df_treino['data_hora'] = df_treino.loc[:,['Date', 'Time']].apply(lambda x: datetime.strptime(x['Date'] + ' ' + x['Time'], '%d/%m/%Y %H:%M'), axis=1)
df_treino = df_treino.sort_values(by='data_hora')
df_treino = df_treino.set_index('data_hora')
#idx = pd.date_range(start=df_treino.index.min(), end=df_treino.index.max(),  freq='0h15min')
#df_treino= df_treino.reindex(idx)
dfx = df_treino[['CO2_room','Relative_humidity_room','Lighting_room','Outdoor_relative_humidity_Sensor','Indoor_temperature_room']].resample('H').mean().iloc[1: , ]
df_treino = pd.DataFrame(dfx, columns = ['CO2_room','Relative_humidity_room','Lighting_room','Outdoor_relative_humidity_Sensor','Indoor_temperature_room'])
df_treino.fillna(method='ffill', inplace=True)

In [ ]:
min(df_treino.index), max(df_treino.index)

In [ ]:
df_treino.info() 

In [ ]:
dfx = pd.DataFrame(df_treino).reset_index()
dfx['data'] =  dfx['data_hora'].apply(lambda x: x.strftime('%Y-%m-%d'))
dfx.groupby('data')['data_hora'].count().reset_index().sort_values('data')

In [ ]:
df_treino = df_treino[df_treino.index.strftime('%Y-%m-%d') !="2012-03-13"]
df_treino = df_treino[df_treino.index.strftime('%Y-%m-%d') !="2012-04-11"]

In [ ]:
df_treino.head(5)

In [ ]:
#dividindo a amostra em treino e teste
dfTreino = df_treino[df_treino.index < datetime(2012,4,8,0,0)]
dfTeste = df_treino[df_treino.index >= datetime(2012,4,8,0,0)]
print(dfTreino.shape, dfTeste.shape)
print(min(dfTreino.index), max(dfTreino.index))
print(min(dfTeste.index), max(dfTeste.index))

In [ ]:
scaler = MinMaxScaler()
treino_values = scaler.fit_transform(dfTreino[['Relative_humidity_room', 'CO2_room' , 'Outdoor_relative_humidity_Sensor', 'Lighting_room']])
teste_values = scaler.fit_transform(dfTeste[['Relative_humidity_room', 'CO2_room' , 'Outdoor_relative_humidity_Sensor', 'Lighting_room']])
dfTreino_tmp = pd.DataFrame(treino_values, columns = ['Relative_humidity_room', 'CO2_room' , 'Outdoor_relative_humidity_Sensor', 'Lighting_room'] )
dfTreino = dfTreino[['Indoor_temperature_room']]
for c in dfTreino_tmp.columns:
  dfTreino[c] = dfTreino_tmp[c].tolist()
 
dfTeste_tmp = pd.DataFrame(teste_values,  columns = ['Relative_humidity_room', 'CO2_room' , 'Outdoor_relative_humidity_Sensor', 'Lighting_room'])
dfTeste = dfTeste[['Indoor_temperature_room']]
for c in dfTeste_tmp.columns:
  dfTeste[c] = dfTeste_tmp[c].tolist()

In [ ]:
dfTreino.head(3)

In [ ]:
stepwise_fit = auto_arima(dfTreino[['Indoor_temperature_room']], start_p=2, start_q=2,
                          max_p=2, max_q=2, m=24,
                          seasonal=True,
                          d=None, trace=True,
                          error_action='ignore',   
                          suppress_warnings=True,  
                          stepwise=True)

In [ ]:
modelo1 = SARIMAX(dfTreino['Indoor_temperature_room'],order=(2,1,2),seasonal_order=(1,0,1,24))
resultado1 = modelo1.fit()
resultado1.summary()




In [ ]:
previsoes1 = resultado1.predict(start=dfTeste.index.min(), end=dfTeste.index.max(), dynamic=False, typ='levels')
dfTeste['previsoes'] = previsoes1
#previsoes1.index = dfTeste.index
#previsoes1['Indoor_temperature_room']=dfTeste['Indoor_temperature_room']

#previsoes1.head(10)

In [ ]:
dfTeste.head(10)

In [ ]:
plotaSeries(None, dfTeste['Indoor_temperature_room'], dfTeste['previsoes'])

In [ ]:
resíduos1 = resultado1.resid

In [ ]:
plot_acf(resíduos1, lags=30)
plot_pacf(resíduos1, lags=30)
plt.show()

In [ ]:
resíduos1.hist()

plt.show()
resíduos1.plot(kind='kde')
plt.show()

In [ ]:
df_teste = pd.read_csv('test.csv')
df_teste['data_hora'] = df_teste.loc[:,['Date', 'Time']].apply(lambda x: datetime.strptime(x['Date'] + ' ' + x['Time'], '%d/%m/%Y %H:%M'), axis=1)
df_teste = df_teste.sort_values(by='data_hora')
df_teste = df_teste.set_index('data_hora')
#dfx = df_treino['Indoor_temperature_room'].resample('H').mean().iloc[1: , ]
#df_teste = pd.DataFrame(dfx, columns = ['Indoor_temperature_room'])
df_teste.fillna(method='ffill', inplace=True)

In [ ]:
true_predictions = resultado1.predict(start=df_teste.index.min(), end=df_teste.index.max(), dynamic=False, typ='levels')
df_teste['Indoor_temperature_room'] = true_predictions
df_teste['Id'] = df_teste['Id'].astype('int')
df_teste[['Id', 'Indoor_temperature_room']].to_csv('hederson_submition.csv', index=False)

In [ ]:
!cat hederson_submition.csv